In [12]:
import pandas as pd

df = pd.read_csv('./output/prev/Under/Dados_Prev_RF.csv')
df.set_index('Open time', inplace = True) # Data time como index
df

,Engolfo,FuCandle,Structure,StructTop,StructBottom,StructPD,VelaBull,ImbLow,ImbHigh,Long,Short,Close,Low,High,ATR
Open time,,,,,,,,,,,,,,,
2017-01-10 20:00:00+03:00,0,0,-1,1.06698,1.03660,0.616853,False,0.00000,0.00000,0,0,1.05534,1.05530,1.05642,0.00392
2017-01-11 00:00:00+03:00,0,0,-1,1.06698,1.03660,0.631007,True,0.00000,0.00000,0,0,1.05577,1.05493,1.05656,0.00376
2017-01-11 04:00:00+03:00,0,0,-1,1.06698,1.03660,0.603687,False,0.00000,0.00000,0,0,1.05494,1.05359,1.05603,0.00366
2017-01-11 08:00:00+03:00,-1,0,-1,1.06698,1.03660,0.555629,False,1.05358,1.05359,0,0,1.05348,1.05300,1.05665,0.00366
2017-01-11 12:00:00+03:00,-1,0,-1,1.06698,1.03660,0.482883,False,0.00000,0.00000,1,0,1.05127,1.04937,1.05358,0.00370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 08:00:00+03:00,-1,0,-1,1.00955,0.99004,0.404408,False,0.00000,0.00000,0,0,0.99793,0.99741,1.00460,0.00411
2022-08-31 12:00:00+03:00,0,0,-1,1.00955,0.99004,0.553562,True,0.00000,0.00000,0,1,1.00084,0.99714,1.00184,0.00416
2022-08-31 16:00:00+03:00,1,0,-1,1.00955,0.99004,0.784726,True,1.00184,1.00355,0,1,1.00535,0.99952,1.00790,0.00446


In [13]:
TAM_STOP =              1                #Tamanho do stop loss em ATRs
RISCO_RETORNO =         3                #Relação risco-retorno 
TAMANHO_LOTE =     100000                #Tamanho do lote padrão
CAPITAL_INICIAL =   10000                #Capital inicial em USD
RISCO_POR_TRADE =   1/100                #Risco por trade em percentual/100
LOTE_MINIMO =        0.01                #Lote mínimo da corretora
LOTE_MAXIMO =         100                #Lote máximo da corretora
FEE =                3.50                #Taxa (one-way) da corretora para um lote em USD
SPREAD =           0.1e-4                #Spread típico em USD (pips/10000); default: 0.1e-4 (0.1 pips)
SLIPPAGE =         0.2e-4                #Slippage típico em USD (pips/10000); default: 0.2e-4 (0.2 pips)
TOTAL_FEES = SPREAD + SLIPPAGE + (2*FEE)/TAMANHO_LOTE #Taxas totais da corretora

In [14]:
#Realiza o trade simulado e retorna o lucro que seria obtido, bem como o ponto onde o trade seria encerrado
def realizarLongTrade(df, timeEntry, money):
    
    #Calcula a posicao do stop, o 'custo' dele (tamanho em pips levando em conta as taxas)
    priceEntry = df['Close'].iloc[timeEntry]
    originalStop = round(df['Low'].iloc[timeEntry] - TAM_STOP * df['ATR'].iloc[timeEntry], 5)
    custoStop = round(abs(priceEntry - originalStop) + TOTAL_FEES, 5)
    
    #Calcula o tamanho da posicao e o take
    size = round(min(max(money/(custoStop*TAMANHO_LOTE), LOTE_MINIMO), LOTE_MAXIMO), 2)
    originalTake = round(df['Close'].iloc[timeEntry] + custoStop * RISCO_RETORNO + TOTAL_FEES, 5)
    
    #Avança no futuro ajustando o stop (trailing) pelo ATR e verifica quando fechar o trade
    priceExit = priceEntry
    timeExit = timeEntry
    currentStop = originalStop
    for j in range(timeEntry+1, len(df)):
        if df['Low'].iloc[j] <= currentStop: #Fecha o trade se foi estopado
            priceExit = currentStop
            timeExit = j
            break  
        elif df['High'].iloc[j] > originalTake: #Realiza o lucro e fecha o trade se atingiu o take
            priceExit = originalTake
            timeExit = j
            break
        else: #Ajusta o stop se possível levando em conta o fundo da vela atual
            newStop = round(df['Low'].iloc[j] - TAM_STOP * df['ATR'].iloc[j], 5)
            if currentStop < newStop:
                currentStop = newStop
    
    #Calcula o lucro do trade
    if timeExit > timeEntry:
        profit = round(priceExit*size*TAMANHO_LOTE - (priceEntry+TOTAL_FEES)*size*TAMANHO_LOTE, 2)
    else:
        profit = 0
    
    #Retorna uma lista com todos os parametros de plotagem
    return profit#[timeEntry, timeExit, priceEntry, originalStop, originalTake, currentStop, priceExit, custoStop, size, profit]
    
    
#Realiza o trade simulado e retorna o lucro que seria obtido, bem como o ponto onde o trade seria encerrado
def realizarShortTrade(df, timeEntry, money):
    
    #Calcula a posicao do stop, o 'custo' dele (tamanho em pips levando em conta as taxas)
    priceEntry = df['Close'].iloc[timeEntry]
    originalStop = round(df['High'].iloc[timeEntry] + TAM_STOP * df['ATR'].iloc[timeEntry], 5)
    custoStop = round(abs(priceEntry - originalStop) + TOTAL_FEES, 5)
    
    #Calcula o tamanho da posicao e o take
    size = round(min(max(money/(custoStop*TAMANHO_LOTE), LOTE_MINIMO), LOTE_MAXIMO), 2)
    originalTake = round(df['Close'].iloc[timeEntry] - custoStop * RISCO_RETORNO - TOTAL_FEES, 5)
    
    #Avança no futuro ajustando o stop (trailing) pelo ATR e verifica quando fechar o trade
    priceExit = priceEntry
    timeExit = timeEntry
    currentStop = originalStop
    for j in range(timeEntry+1, len(df)):
        if df['High'].iloc[j] >= currentStop: #Fecha o trade se foi estopado
            priceExit = currentStop
            timeExit = j
            break  
        elif df['Low'].iloc[j] < originalTake: #Realiza o lucro e fecha o trade se atingiu o take
            priceExit = originalTake
            timeExit = j
            break
        else: #Ajusta o stop se possível levando em conta o fundo da vela atual
            newStop = round(df['High'].iloc[j] + TAM_STOP * df['ATR'].iloc[j], 5)
            if currentStop > newStop:
                currentStop = newStop
    
    #Calcula o lucro do trade
    if timeExit > timeEntry:
        profit = round(priceEntry*size*TAMANHO_LOTE - (priceExit+TOTAL_FEES)*size*TAMANHO_LOTE, 2)
    else:
        profit = 0
    
    #Retorna uma lista com todos os parametros de plotagem
    return profit#[timeEntry, timeExit, priceEntry, originalStop, originalTake, currentStop, priceExit, custoStop, size, profit]

In [15]:
def realizarTodosTrades(df):
    granaAtual = CAPITAL_INICIAL
    df['Equity'] = CAPITAL_INICIAL
    df['Profit'] = 0

    #Geramos todos os trades
    for i in range(0, len(df)):        
        if df['Long'].iloc[i] == 1:
            profit = realizarLongTrade(df, i, round(granaAtual*RISCO_POR_TRADE, 2))
            df.at[df.index[i], 'Profit'] = 1 if profit > 0 else 0
            granaAtual += profit
            if (granaAtual < 0):
                break
        elif df['Short'].iloc[i] == 1:
            profit = realizarShortTrade(df, i, round(granaAtual*RISCO_POR_TRADE, 2))
            df.at[df.index[i], 'Profit'] = 1 if profit > 0 else 0
            granaAtual += profit
            if (granaAtual < 0):
                break
        df.at[df.index[i], 'Equity'] = granaAtual
    return round(granaAtual, 2)

In [16]:
profit = realizarTodosTrades(df)
df1 = df.drop(df[df['Profit'] < 1].index)
df0 = df.drop(df[df['Profit'] > 0].index)
profit

48380.68

In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots 

In [18]:
TICKET = 'EURUSD'                   #Moeda (par)
TIMEFRAME = 240                     #Tempo gráfico
INICIO = '1999-01-01'               #Data inicial
FINAL = '2024-04-01'                #Data final

In [19]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.2, row_width=[0.2, 0.7])
fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], showlegend=False, line=dict(width=1.0), opacity=1.0, 
            increasing_fillcolor='#dadce3', decreasing_fillcolor="#5d606b", increasing_line_color='#434651', decreasing_line_color='#434651'), row=1, col=1)  


fig.add_trace(go.Scatter(x=df.index, y=df['Equity'], name='Equity', showlegend=False, opacity=1.0, mode='lines', line=dict(color="#787B86", width=1.0, dash='solid')), row=2, col=1)

fig.update_yaxes(type='linear', row=1, col=1)
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    
fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=TICKET + str(TIMEFRAME) + '   -   ' + INICIO + ' a ' + FINAL, xaxis_tickfont_size=12, xaxis=dict(titlefont_size=14, tickfont_size=12), yaxis=dict(title='Price', 
            titlefont_size=14, tickfont_size=12), height=850, autosize=True, margin=dict(l=0,r=20, b=20, t=40, pad=0), paper_bgcolor='white', template='simple_white')
fig.show()

KeyError: 'Open'